![](https://i.imgur.com/qkg2E2D.png)

# Natural Language Processing

## Assignment 002 - POS Tagging with Universal Dependencies

> Notebook by:
> - NLP Course Stuff

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
|  |  |    | |
| 0.1.000 | 01/05/2024 | Course stuff | First version                                                      |
|         |            |             |                                                                    |

## Overview
In this assignment, you will train and evaluate a Part-of-Speech (POS) tagger using data from the Universal Dependencies (UD) project. POS taggers assign parts of speech to each word in a sentence, such as noun, verb, adjective, etc., which are crucial for many natural language processing tasks.

## Dataset
Utilize the English Web Treebank from the Universal Dependencies project. You can access and explore the dataset [here](https://universaldependencies.org/). For a better understanding of the project and the data format, visit the [introduction page](https://universaldependencies.org/introduction.html).

## Tasks Overview

#### Part 1: Dataset Preparation
- **Objective**: Get the Universal Dependencies dataset ready for the tagging tasks.
- **Activities**: Download, preprocess, and format the dataset.

#### Part 2: HMM Tagger
- **Objective**: Create and assess a POS tagger using the Hidden Markov Model.
- **Activities**: Construct, train, and evaluate the HMM tagger.

#### Part 3: Feed-Forward Neural Network Tagger
- **Objective**: Build a POS tagger using a feed-forward neural network with word embeddings.
- **Activities**: Develop and train the model in PyTorch, then evaluate its effectiveness.

#### Part 4: NLTK MEMM Tagger
- **Objective**: Implement and test a MEMM-based POS tagger using NLTK.
- **Activities**: Train the MEMM tagger, then evaluate its performance.

#### Part 5: Models' Comparison
- **Objective**: Evaluate and contrast the performance of different models.
- **Activities**: Address two open-ended questions.


## Your Implementation

Please create a local copy of this template Colab's Notebook:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1FfXDvRMALIsd-IzPdf_Fn92OLVHjSY9I#scrollTo=JCFoQxO0H0yk)

The assignment's instructions are there; follow the notebook.

## Submission
- **Notebook Link**: Add the URL to your assignment's notebook in the `notebook_link.txt` file, following the format provided in the example.
- **Access**: Ensure the link has edit permissions enabled to allow modifications if needed.
- **Deadline**: <font color='green'>21/05/2024</font>.
- **Platform**: Continue using GitHub for submissions. Push your project to the team repository and monitor the test results under the actions section.

Good Luck 🤗


In [1]:
# Prerequisite: Install the conllutils library before proceeding with the tasks
# !pip install --q conllutils

In [2]:
# Standard Library Imports
import os
import random
import operator
import json
from typing import List, Tuple, Dict
from collections import defaultdict

# Data Handling and Numerical Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
# from google.colab import files

# Machine Learning and Evaluation Libraries
from sklearn.metrics import classification_report, precision_recall_fscore_support

# Natural Language Processing Libraries
import nltk
from nltk.tag import tnt
from nltk.metrics import ConfusionMatrix, precision, recall, f_measure

# Deep Learning Libraries (PyTorch)
import torch as th
import torch.nn as nn
import torch.optim as optim

# CoNLL Utilities for Data Handling
import conllutils
import gensim


# Part 1 - Dataset Preparation





For each package you use, set the random seed to 42.

In [3]:
SEED = 42
# Set the random seed for Python
random.seed(SEED)

# Set the random seed for numpy
np.random.seed(SEED)

# Set the random seed for pytorch
th.manual_seed(SEED)

# If using CUDA (for GPU operations)
th.cuda.manual_seed(SEED)

## Instructions

### Step 1: Access the Dataset
The GUM dataset, which is part of the English corpora under Universal Dependencies, is specifically curated for academic and research purposes. You can download the dataset directly from the following GitHub repository:

[UD English-GUM](https://github.com/UniversalDependencies/UD_English-GUM)

In [4]:
!git clone https://github.com/UniversalDependencies/UD_English-GUM

fatal: destination path 'UD_English-GUM' already exists and is not an empty directory.


### Step 2: Reading the Data
We will use the (train/dev/test) files:


```
UD_English-GUM/en_gum-ud-train.conllu
UD_English-GUM/en_gum-ud-dev.conllu
UD_English-GUM/en_gum-ud-test.conllu
```

## CoNLL-U Format
They are all formatted in the CoNLL-U format. You may read about it [here](https://universaldependencies.org/format.html). There is a utility library **conllutils**, which can help you read the data into the memory. It has already been installed and imported above.

## Task: Create a Read Data Function

### Function Specification
Create a function named `read_data` that:
- Takes a file path to a `.conllu` file as input.
- Returns a list of lists, where each inner list represents a sentence.
- Each sentence is composed of tuples containing the word ('form') and its corresponding Universal POS tag ('upos').

The word is located in the column named 'form' and the POS tag in the column named 'upos' of the CoNLL-U format.

In [5]:
DataType = list[list[tuple[str, str]]]

In [6]:
from conllutils import pipe
UNK = '<<UNK>>'
def read_data(filepath: str) -> DataType:
    """
    Reads a CoNLL-U formatted file and extracts sentences as lists of (word, POS tag) tuples.

    Args:
    filepath (str): The path to the .conllu file to be read.

    Returns:
    List[List[Tuple[str, str]]]: A list of sentences, where each sentence is a list of tuples containing the word and its POS tag.
    """
    output = []
    
    NUM_REGEX = r'[0-9]+|[0-9]+\.[0-9]+|[0-9]+[0-9,]+'
    NUM_FORM = '__number__'
    p = pipe()
    p.only_words()
    # p.only_universal_deprel()
    # p.upos_feats()
    # p.lowercase('form')
    # p.replace('form', NUM_REGEX, NUM_FORM)

    train_data = pipe().read_conllu(filepath).pipe(p).collect()

    for s in train_data:
        pos_list = []
        for x in s:
            try:
                upos = x.upos
            except:
                upos = UNK
            if x.form == "_":
                x.form = UNK
            pos_list.append((x.form, upos))
        output.append(pos_list)

    return output

In [7]:
# Run this block once `read_data` is implemented.
train_dataset = read_data("UD_English-GUM/en_gum-ud-train.conllu")
dev_dataset = read_data("UD_English-GUM/en_gum-ud-dev.conllu")
test_dataset = read_data("UD_English-GUM/en_gum-ud-test.conllu")

## Task: Create a Vocabulary Generation Function

### Function Specification
Create a function named `generate_vocabs` for mapping words and tags into unique numbers so that we can use them in the tagging algorithms you will implement below. The function:
- Takes a list of sentences as input, where each sentence is composed of tuples containing a word and its corresponding Universal POS tag.
- Returns a tuple of two dictionaries:
  - The first dictionary maps each unique word to a unique integer.
  - The second dictionary maps each unique POS tag to a unique integer.

Each word and POS tag should be mapped starting from 0, with each new word or tag encountered receiving the next sequential integer. This function is essential for converting textual data into a numerical format that can be used by tagging algorithms.



In [8]:
def generate_vocabs(datasets: list[DataType]) -> tuple[dict[str, int], dict[str, int]]:
    """
    Generates vocabularies mapping words and tags to unique indices from a list of datasets.

    Args:
    datasets (list): A list of datasets where each dataset contains sentences formatted as lists of (word, tag) tuples.

    Returns:
    Tuple[dict[str, int], dict[str, int]]: A tuple of two dictionaries:
        - The first dictionary maps each unique word to a unique integer.
        - The second dictionary maps each unique POS tag to a unique integer.
        Each dictionary includes a special '<<UNK>>' entry mapped to 0 to handle unknown words or tags.
    """
    words_vocab = {"<<UNK>>": 0}
    tags_vocab = {"<<UNK>>": 0}
    word_index = 1
    tag_index = 1
    for dataset in datasets:
        for sentence in dataset:
            for word, tag in sentence:
                if word not in words_vocab:
                    words_vocab[word] = word_index
                    word_index += 1
                if tag not in tags_vocab:
                    tags_vocab[tag] = tag_index
                    tag_index += 1

    return words_vocab, tags_vocab


In [9]:
# Run this block once `generate_vocabs` is implemented.
words_vocab, tags_vocab = generate_vocabs([train_dataset, dev_dataset, test_dataset])
reversed_words_vocab = {v: k for k, v in words_vocab.items()}
reversed_tags_vocab = {v: k for k, v in tags_vocab.items()}

# Part 2 - HMM Tagger

### Task Description
Implement a class `HMMTagger` to perform POS tagging using a Hidden Markov Model (HMM).

### Class Methods
- `fit`: This method should compute the transition probabilities matrix (A), emission probabilities matrix (B), and initial state probabilities vector (Pi) based on the training data. These matrices should reflect probabilities of transitions between tags, emissions of words given tags, and initial tag probabilities, respectively.
- `inference`: Implement this method to predict the best tag sequence for a given input sentence using the Viterbi decoding algorithm. The Viterbi algorithm is provided below.
### Additional Guidance
1. **Use of Vocabularies**: Utilize the vocabularies generated in Part 1. These should include a special entry for unknown words and tags (`<<UNK>>` at index 0). The indices in your vocabularies will correspond to the rows and columns of your A, B, and Pi matrices (np.array).
2. **Smoothing**: Apply Add-One Smoothing to all probability calculations to avoid zero probabilities. This technique adjusts the frequency counts for each observed event by adding one to each count.
3. **Word Conversion**: During inference, convert each word of the input sentence into its corresponding index using the word vocabulary. If a word is not found, use the index for `<<UNK>>`.

### Implementation Tips
- You can use the vocab dictionaries directly, no need to pass them as a parameter to the functions.
- You may add functions to `HMMTagger` as needed.
- Ensure that your A, B, and Pi matrices handle unseen words/tags gracefully using the `<<UNK>>` index.


In [10]:
class HMMTagger:
    def __init__(self):
        """
        Initializes the HMMTagger class with necessary attributes.
        """
        self._π = np.zeros(len(tags_vocab))  # Initial state probabilities
        self._A = np.zeros((len(tags_vocab), len(tags_vocab)))  # Transition probabilities
        self._B = np.zeros((len(tags_vocab), len(words_vocab)))  # Emission probabilities

    def fit(self, dataset: DataType):
        """
        Trains the HMM model on the provided dataset.

        Args:
            dataset (list): The training dataset containing sentences as lists of (word, tag) tuples.
        """
        # Compute initial state probabilities
        for sentence in dataset:
            self._π[tags_vocab[sentence[0][1]]] += 1

        self._π = (self._π + 1) / (len(dataset) + len(tags_vocab))

        # Compute transition probabilities
        for sentence in dataset:
            for i in range(len(sentence) - 1):
                tag1 = tags_vocab[sentence[i][1]]
                tag2 = tags_vocab[sentence[i + 1][1]]
                self._A[tag1, tag2] += 1

        self._A = (self._A + 1) / (np.sum(self._A, axis=1, keepdims=True) + len(tags_vocab))

        # Compute emission probabilities
        for sentence in dataset:
            for word, tag in sentence:
                word_index = words_vocab.get(word, 0)
                tag_index = tags_vocab[tag]
                self._B[tag_index, word_index] += 1

        # Apply add-one smoothing and normalize
        self._B = (self._B + 1) / (np.sum(self._B, axis=1, keepdims=True) + len(words_vocab))

    def inference(self, sentence: list) -> list[Tuple[str, str]]:
        """
        Predicts the best tag sequence for a given input sentence using the Viterbi decoding algorithm.

        Args:
            sentence (list): The sentence to tag, as a list of words.

        Returns:
            List[Tuple[str, str]]: Each word in the input sentence paired with its predicted tag.
        """
        # Initialize the Viterbi algorithm
        T = len(sentence)
        N = len(tags_vocab)
        viterbi = np.zeros((T, N))
        backpointer = np.zeros((T, N), dtype=int)

        # Initialize the first row in the Viterbi matrix
        for s in range(N):
            viterbi[0, s] = self._π[s] * self._B[s, words_vocab.get(sentence[0], 0)]

        # Iterate over the rest of the words in the sentence
        for t in range(1, T):
            for s in range(N):
                viterbi[t, s] = np.max(viterbi[t - 1] * self._A[:, s]) * self._B[s, words_vocab.get(sentence[t], 0)]
                backpointer[t, s] = np.argmax(viterbi[t - 1] * self._A[:, s])

        # Backtrack to find the best tag sequence
        best_path = [np.argmax(viterbi[T - 1])]
        for t in range(T - 1, 0, -1):
            best_path.append(backpointer[t, best_path[-1]])
        best_path.reverse()

        return [(sentence[i], reversed_tags_vocab[best_path[i]]) for i in range(T)]


### Optional Task: Implement the Viterbi Algorithm
Implement the `viterbi` function to perform POS tagging using the Viterbi decoding algorithm. This algorithm finds the most probable sequence of hidden states (POS tags in this case) given a sequence of observations (words in the sentence).

### Instructions
- **Pre-Implemented Code**: We provide a pre-implemented version of the Viterbi algorithm for your convenience. This implementation is fully functional and can be used directly in your HMM tagger.
  
- **Implementation Challenge**: Although a pre-implemented version is available, we encourage you to implement the Viterbi algorithm yourself. Doing so will help you understand the dynamics of dynamic programming in the context of POS tagging. Follow the pseudocode provided in the lecture slides to develop your own version of the algorithm.

### Steps for Implementation
1. **Understand the Pseudocode**: Review the pseudocode provided in the slides from the class. Ensure you understand each step of the algorithm, including how the probabilities are updated and the backtracking process to recover the state sequence.
  
2. **Implement the Function**: Using the pseudocode as a guide, write your own `viterbi` function. Consider the matrices (A, B, and Pi) you prepared in the HMMTagger class as inputs along with the sequence of observations (word indices for a sentence).
  
3. **Test Your Implementation**: After implementing the function, test it with known inputs to ensure it produces the correct sequence of tags. Compare the results with those obtained from the pre-implemented version to validate your implementation.

### Additional Tips
- **Handle Edge Cases**: Consider edge cases such as very short sentences, sentences containing many unknown words, and varying sentence structures.
- **Optimization**: Once your basic implementation is correct, think about potential optimizations to improve the efficiency of your code, especially if you are processing large datasets.


In [11]:
def viterbi(word_list: list, A: np.ndarray, B: np.ndarray, Pi: np.ndarray):
    """
    Executes the Viterbi algorithm to find the most likely state sequence given a sequence of observations.

    The Viterbi algorithm is a dynamic programming algorithm used to compute the most likely sequence of hidden states
    (called the Viterbi path) that results in a sequence of observed events, especially in the context of Markov information sources and hidden Markov models (HMM).

    Args:
        word_list (list): A list of indices corresponding to observed words.
        A (numpy.ndarray): The state transition probability matrix of shape (num_states, num_states) where A[i][j] is the probability of transitioning from state i to state j.
        B (numpy.ndarray): The emission probability matrix of shape (num_states, num_vocabulary) where B[i][j] is the probability of emitting symbol j from state i.
        Pi (numpy.ndarray): The initial state probability vector of length num_states where Pi[i] is the probability of starting in state i.

    Returns:
        list: The most likely sequence of states (as indices) for the given sequence of observations.

    The function uses three main steps: initialization, recursion, and termination:
    - **Initialization**: Set the initial probabilities of being in each state.
    - **Recursion**: For each subsequent observation, compute probabilities of each state based on the observation and transition probabilities from previous states.
    - **Termination**: Backtrace to determine the most probable path through the state space.
    """
    # Number of states
    num_states = len(A)
    # Length of the observed sequence
    T = len(word_list)

    # Create the path probability matrix V
    V = [[0 for _ in range(num_states)] for _ in range(T)]
    # Create a path backpointer matrix to store the argmax indices
    path = [[0 for _ in range(num_states)] for _ in range(T)]

    # Initialization step
    for s in range(num_states):
        V[0][s] = Pi[s] * B[s][word_list[0]]
        path[0][s] = 0

    # Recursion step
    for t in range(1, T):
        for s in range(num_states):
            # Find the state with the maximum probability to reach state s
            max_tr_prob = V[t-1][0] * A[0][s]
            prev_state_selected = 0
            for prev_state in range(1, num_states):
                tr_prob = V[t-1][prev_state] * A[prev_state][s]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_state_selected = prev_state
            # Multiply the max probability by the probability of observing the symbol at state s
            max_prob = max_tr_prob * B[s][word_list[t]]
            V[t][s] = max_prob
            path[t][s] = prev_state_selected

    # Termination step
    # Find the best path by looking for the maximum probability in the last column
    opt = []
    max_prob = -1
    best_last_state = 0
    for s in range(num_states):
        if V[T-1][s] > max_prob:
            max_prob = V[T-1][s]
            best_last_state = s
    opt.append(best_last_state)

    # Follow the back pointers to decode the best path
    previous = best_last_state
    for t in range(T-1, 0, -1):
        opt.insert(0, path[t][previous])
        previous = path[t][previous]

    return opt

In [12]:
# Example for using Viterbi algorithm (from the course slides)
A = np.array([[0.3, 0.7], [0.2, 0.8]])
B = np.array([[0.1, 0.1, 0.3, 0.5], [0.3, 0.3, 0.2, 0.2]])
Pi = np.array([0.4, 0.6])

viterbi([0, 3, 2, 0], A, B, Pi)

[1, 1, 1, 1]

## Train & Evaluate

## Task: Implement the Train and Evaluate Tagger Function

### Function Specification
Create a function named `train_evaluate_tagger` that combines the training and evaluation processes for a tagging algorithm. This integrated function should:
- **Input**:
  - **tagger** (`HMMTagger` or `EmbeddingsTagger`): The POS tagger to be trained and evaluated.
  - **train_dataset** (`List[List[Tuple[str, str]]]`): The dataset on which the tagger will be trained. Training is executed only if the `train` flag is set to True.
  - **eval_dataset** (`List[List[Tuple[str, str]]]`): The dataset used for evaluating the tagger's performance.
  - **train** (`bool`): A boolean flag indicating whether the training phase should be executed. If set to True, the `fit` method of the tagger will be called before evaluation.
- **Functionality**:
  - Train the tagger using the `train_dataset`.
  - Evaluate the trained tagger on the `test_dataset` to compute performance metrics.
  - Return the following evaluation metrics: accuracy, precision, recall, and F1-score.
- **Outputs**: A tuple consisting of three elements: precision, recall, F1-score, as returned by the `precision_recall_fscore_support` function from the `sklearn.metrics` module, when called with the parameter `average="macro"`.
* Note the the support metric is missed (Why?).


### Evaluation Metrics
Upon execution, the `train_evaluate_tagger` function provides the following metrics:
- **Macro-average Precision, Recall, and F1 Score**: These scores are calculated over all tags to assess the overall effectiveness of the tagger in recognizing the correct tags across different types of words.
- **Performance Breakdown by Tag**: Detailed metrics for each tag, helping to identify which tags are most and least accurately predicted.
- **Overall Word-Level Accuracy**: Measures the percentage of words correctly tagged by the tagger across the entire evaluation dataset.



In [13]:
def train_evaluate_tagger(tagger, train_dataset: DataType=train_dataset, eval_dataset: DataType=test_dataset, train=True):
    """
    Trains (optional) and evaluates a POS tagger, returning performance metrics.

    This function trains the given tagger if specified, and evaluates it on a provided dataset.
    It calculates and returns the macro-average precision, recall, and F1-score.

    Args:
        tagger (HMMTagger or EmbeddingsTagger): The POS tagger to be trained and evaluated.
        train_dataset (List[List[Tuple[str, str]]]): The dataset to train the tagger.
        eval_dataset (List[List[Tuple[str, str]]]): The dataset to evaluate the tagger.
        train (bool): A flag indicating whether the tagger should be trained.

    Returns:
        tuple: A tuple containing average precision, recall, and F1-score.

    The function also prints a classification report for detailed performance analysis.
    """

    true_tags = []
    predicted_tags = []

    if train:
        tagger.fit(train_dataset)

    for sentence in eval_dataset:
        words = [word for word, _ in sentence]
        true_tags.extend([tag for _, tag in sentence])
        predicted_tags.extend([tag for _, tag in tagger.inference(words)])

    # Compute precision, recall, F1-score, and support for each class
    precision, recall, f1_score, support = precision_recall_fscore_support(true_tags, predicted_tags, average=None)

    # Display classification report
    report = classification_report(true_tags, predicted_tags)
    print("Classification Report:")
    print(report)

    # Return the macro-average metrics
    avg_precision, avg_recall, avg_f1, _ = precision_recall_fscore_support(true_tags, predicted_tags, average='macro')

    return avg_precision, avg_recall, avg_f1


In [14]:
# Train (optional) and Evaluate your HMMTagger here
hmmTagger = HMMTagger()
precision_hmm, recall_hmm, f1_hmm = train_evaluate_tagger(hmmTagger)

/Users/bensa/miniconda3/envs/NLP/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/bensa/miniconda3/envs/NLP/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/bensa/miniconda3/envs/NLP/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/

Classification Report:
              precision    recall  f1-score   support

     <<UNK>>       0.00      0.00      0.00         0
         ADJ       0.79      0.74      0.77      1621
         ADP       0.86      0.97      0.91      2480
         ADV       0.85      0.74      0.79      1114
         AUX       0.80      0.96      0.87      1188
       CCONJ       0.97      0.97      0.97       839
         DET       0.82      0.97      0.89      2111
        INTJ       0.70      0.63      0.66       163
        NOUN       0.83      0.82      0.82      4238
         NUM       0.72      0.65      0.68       440
        PART       0.86      0.79      0.82       519
        PRON       0.82      0.97      0.89      1745
       PROPN       0.83      0.47      0.60      1628
       PUNCT       0.92      1.00      0.96      3027
       SCONJ       0.80      0.54      0.65       340
         SYM       0.50      0.29      0.36        35
        VERB       0.87      0.76      0.81      2471
    

/Users/bensa/miniconda3/envs/NLP/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Part 3 - POS Tagging with Pre-trained Word Embeddings

### Task Overview
Develop a feed-forward neural network for Part-of-Speech (POS) tagging using the PyTorch framework. This tagger leverages pre-trained word embeddings from the word2vec Google News dataset, enhancing the semantic understanding of words compared to traditional tagging methods.

### Task 1: Initialization of Embeddings (`init_embeddings`)
Create an `init_embeddings` method within the `EmbeddingsTagger` class to load and set up pre-trained word embeddings:
- Load the Google News word vectors from a specified file path.
- Initialize a matrix to hold these embeddings where each word in your vocabulary is represented by a vector. For words not in the pre-trained model, initialize their vectors randomly.
-  <font color='red'>**NOTE:** Before you start implementing the init_embeddings method, make sure to create your own copy of the pre-trained embeddings (.bin file) in your Google Drive from [the following Google Drive folder](https://drive.google.com/drive/folders/1-HcSBfqaX0PCFiT8TsiYjFZRQJRm2R5v?usp=sharing). You will need to use the gensim library to load this file.</font>

### Task 2: Implementation of `EmbeddingsTagger` Class
Extend PyTorch's `nn.Module` to implement the `EmbeddingsTagger` class. This class should utilize the embeddings matrix and include:
- An embedding layer that is initialized with the pre-trained embeddings.
- A linear layer to combine embeddings of the current and previous words with a one-hot encoded previous tag to predict the current tag.
- A `forward` method that outlines the data flow through the network.

### Task 3: Model Training and Evaluation (`fit` and `inference` Methods)
Implement training and inference functionalities within the `EmbeddingsTagger` class:
- **Train Method (`fit`)**: Set up the model training using the specified training dataset. This includes iterating through the dataset, applying the model to predict tags, and updating model parameters based on the loss computation.
- **Inference Method (`inference`)**: Configure the model to predict tags on a new dataset, assessing the model's effectiveness on unseen data.

### Model Performance Monitoring
Utilize the `train_evaluate_tagger` function to oversee the training process and evaluate the model:
- Configure this function to handle model training with appropriate optimizer and loss settings.
- Monitor and report the model's performance metrics during training and on a test dataset to ensure effective tagging.


### Google News pre-trained embeddings


In [15]:
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [22]:
from tqdm import tqdm

class EmbeddingsTagger(nn.Module):
    def __init__(self, device="cpu"):
        """
        Initializes an embeddings-based POS tagger that uses word embeddings from a pre-trained model.

        Args:
            device (str): The device (cpu or cuda) the model should operate on for tensor operations.
        """
        self.device = device
        super(EmbeddingsTagger, self).__init__()
        self.embedding_dim = 300  # Dimension of Google News embeddings
        self.tagset_size = len(tags_vocab)
        self.word_embeddings = self.init_embeddings()
        self.to(device)


        # The input dimension to the linear layer is twice the embedding_dim (for two words)
        # plus tagset_size (for the one-hot encoded previous tag)
        self.linear_layer = nn.Linear(2 * self.embedding_dim + self.tagset_size, self.tagset_size).to(self.device)


    def init_embeddings(self) -> nn.Embedding:
        """
        Loads word embeddings from a pre-trained Google News model and initializes an embedding layer.

        Returns:
            nn.Embedding: A PyTorch embedding layer with the pre-trained word embeddings.
        """
        # Specify the path to the Google News model in your Google Drive
        # path_to_google_news_vectors = '/content/drive/My Drive/GoogleNews-vectors-negative300.bin'
        path_to_google_news_vectors = 'GoogleNews-vectors-negative300.bin'

        # Load Google News Vectors
        print("Loading Google News Vectors...")
        model = gensim.models.KeyedVectors.load_word2vec_format(path_to_google_news_vectors, binary=True)

        # Prepare a matrix to hold the embeddings
        embedding_matrix = np.zeros((len(words_vocab), self.embedding_dim))  # Ensure 'words_vocab' maps words to indices, 300 is the dimension of embeddings

        # Fill the matrix with embeddings from the pre-trained model
        for word, index in tqdm(words_vocab.items(), desc="Loading Embeddings"):
            if word in model:
                embedding_matrix[index] = model[word]
            else:
                # For words not in the pre-trained model, initialize their vectors randomly
                embedding_matrix[index] = np.random.uniform(-0.25, 0.25, self.embedding_dim)

        # Initialize the embedding layer with the pre-trained embeddings
        return nn.Embedding.from_pretrained(th.FloatTensor(embedding_matrix), freeze=True)

    def forward(self, current_word_index: int, previous_word_index: int, prev_tag_one_hot: th.Tensor):
        """
        Forward pass of the tagger to compute logits for each tag.

        Args:
            current_word_index (int): Index of the current word in the vocabulary.
            previous_word_index (int): Index of the previous word in the vocabulary.
            prev_tag_one_hot (th.tensor): One-hot encoded tensor of the previous tag.

        Returns:
            th.tensor: Logits for each tag.
        """
        current_word_embedding = self.word_embeddings(th.LongTensor([current_word_index]).to(self.device))
        previous_word_embedding = self.word_embeddings(th.LongTensor([previous_word_index]).to(self.device))
        tag_scores = self.linear_layer(th.cat((current_word_embedding, previous_word_embedding, prev_tag_one_hot[None, :]), 1))
        return tag_scores


    def fit(self, dataset=list(train_dataset), epochs=1):
        """
        Trains the tagger on the provided dataset for a specified number of epochs.

        Args:
            dataset (list): The dataset to train the model on.
            epochs (int): Number of training epochs.
        """
        loss_function = nn.NLLLoss()
        optimizer = optim.Adam(self.parameters(), lr=0.01)

        # preparing instances for training
        instances = []

        for sentence in dataset:
            for i in range(len(sentence)):
                current_word = sentence[i][0]
                previous_word = sentence[i - 1][0] if i > 0 else UNK
                prev_tag_one_hot = th.zeros(self.tagset_size)
                prev_tag_one_hot[tags_vocab[sentence[i - 1][1]]] = 1 if i > 0 else 0
                prev_tag_one_hot = prev_tag_one_hot.to(self.device)
                target_tag = th.LongTensor([tags_vocab[sentence[i][1]]]).to(self.device)
                instances.append((words_vocab[current_word], words_vocab[previous_word], prev_tag_one_hot, target_tag))

        loss_c = 0
        for epoch in range(epochs):
            for index, (current_word, previous_word, prev_tag_one_hot, target_tag) in enumerate(instances):
                self.zero_grad()
                tag_scores = self(current_word, previous_word, prev_tag_one_hot)
                loss = loss_function(tag_scores, target_tag)
                loss.backward()
                optimizer.step()
                loss_c += loss.item()
                if index % 1000 == 0:
                    print(f"Epoch: {epoch}, Instance: {index}, Loss: {loss_c / 1000}")
                    loss_c = 0




    def inference(self, sentence: list) -> List[Tuple[str, str]]:
        """
        Predicts the tags for each word in a given sentence.

        Args:
            sentence (list): The sentence to tag, given as a list of words.

        Returns:
            list: A list of tuples containing each word and its predicted tag.
        """

        predicted_tags = []
        prev_tag_one_hot = th.zeros(self.tagset_size)

        for i in range(len(sentence)):
            current_word = sentence[i]
            previous_word = sentence[i - 1] if i > 0 else UNK
            prev_tag_one_hot = th.zeros(self.tagset_size)
            if i > 0:
                prev_tag_one_hot[tags_vocab[predicted_tags[-1]]] = 1
            prev_tag_one_hot = prev_tag_one_hot.to(self.device)
            current_word_index = words_vocab.get(current_word, 0) 
            previous_word_index = words_vocab.get(previous_word, 0)
            tag_scores = self(current_word_index, previous_word_index, prev_tag_one_hot)
            _, predicted_tag_index = th.max(tag_scores, 1)
            predicted_tags.append(reversed_tags_vocab[predicted_tag_index.item()])

        return list(zip(sentence, predicted_tags))


embeddings_tagger = EmbeddingsTagger()
true_tags = []
predicted_tags = []


for sentence in test_dataset:
    words = [word for word, _ in sentence]
    true_tags.extend([tag for _, tag in sentence])
    predicted_tags.extend([tag for _, tag in embeddings_tagger.inference(words)])


Loading Google News Vectors...


Loading Embeddings: 100%|██████████| 19873/19873 [00:08<00:00, 2245.53it/s]


In [23]:
# Initialize the model with pre-trained embeddings
# device = th.device("cuda")
embeddings_tagger = EmbeddingsTagger()

Loading Google News Vectors...


Loading Embeddings: 100%|██████████| 19873/19873 [00:07<00:00, 2677.18it/s]


In [27]:
precision_embeddings, recall_embeddings, f1_embeddings = train_evaluate_tagger(embeddings_tagger)

print("HMM Tagger Performance:")
print(f"Precision: {precision_hmm:.4f}, Recall: {recall_hmm:.4f}, F1-Score: {f1_hmm:.4f}")

Epoch: 0, Instance: 0, Loss: 3.874577581882477e-05
Epoch: 0, Instance: 1000, Loss: -20.205977838669902
Epoch: 0, Instance: 2000, Loss: -60.37795838558674
Epoch: 0, Instance: 3000, Loss: -90.47261043596268
Epoch: 0, Instance: 4000, Loss: -119.91439978659153
Epoch: 0, Instance: 5000, Loss: -166.2151823825836
Epoch: 0, Instance: 6000, Loss: -195.12943202623725
Epoch: 0, Instance: 7000, Loss: -226.24927525424957
Epoch: 0, Instance: 8000, Loss: -259.4020686798096
Epoch: 0, Instance: 9000, Loss: -311.9010826072693
Epoch: 0, Instance: 10000, Loss: -348.2499318642616
Epoch: 0, Instance: 11000, Loss: -365.99608721256254
Epoch: 0, Instance: 12000, Loss: -416.49394344329835
Epoch: 0, Instance: 13000, Loss: -450.18002555179595
Epoch: 0, Instance: 14000, Loss: -454.15850499916075
Epoch: 0, Instance: 15000, Loss: -471.99215736579896
Epoch: 0, Instance: 16000, Loss: -456.0799918870926
Epoch: 0, Instance: 17000, Loss: -561.885618801117
Epoch: 0, Instance: 18000, Loss: -549.5815168876647
Epoch: 0, Inst


# Part 4 - NLTK Tagger

### Overview
In this final part of the assignment, you will evaluate the performance of the HMM-based and feed-forward taggers you developed against a Maximum Entropy Markov Model (MEMM) tagger implemented using the Natural Language Toolkit (NLTK), a popular NLP library. Perform comparison should cover the test dataset.


#### Step 1: Training the MEMM Tagger


In [19]:
 # TO DO ----------------------------------------------------------------------
 # TO DO ----------------------------------------------------------------------

#### Step 2: Evaluation
- Evaluate the trained MEMM tagger on  the test dataset.
- Calculate performance metrics such as accuracy, and F1-score. NLTK provides utilities that can help compute these metrics efficiently.
- Save & Print the eveluation scores.


In [20]:
 # TO DO ----------------------------------------------------------------------
 # Accurcy
accuracy_tnt_pos_tagger = # Complete
 # TO DO ----------------------------------------------------------------------
 accuracy_tnt_pos_tagger

In [21]:
 # TO DO ----------------------------------------------------------------------
 # F-measure for each tag

 # TO DO ----------------------------------------------------------------------

# Evaluatoin and Comparison
Compare the results obtained from the MEMM tagger with those from your HMM and feed-forward neural network taggers.

This part won't be tested by the autograder.

#### Discuss the following:



* <font color='red'>(**?**)</font> Which tagger performed best on each dataset and why?
* <font color='green'>(**Answer**)</font> :

* <font color='red'>(**?**)</font> What are the strengths and weaknesses of each approach (HMM, feed-forward neural network, MEMM) in terms of training time, accuracy, and generalizability?
* <font color='green'>(**Answer**)</font> :

# Testing
Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [26]:
####################
# PLACE TESTS HERE #
def test_read_data_data_types():
    data = read_data("UD_English-GUM/en_gum-ud-train.conllu")
    result = {
        'is_data_list': type(data) == list,
        'is_data_first_element_list': type(data[0]) == list,
        'is_data_first_element_first_item_tuple': type(data[0][0]) == tuple
    }
    return result

def test_read_data_len_train_data():
    return {
        'train_data_length': len(read_data("UD_English-GUM/en_gum-ud-train.conllu")),
    }

def test_generate_vocabs():
    return {
        'vocab_size': len(words_vocab),
        'num_tags': len(tags_vocab)
    }

def test_hmm():
    return {
        'precision': round(precision_hmm, 2),
        'recall': round(recall_hmm, 2),
        'f1': round(f1_hmm, 2),
    }

def test_embeddings_model():
    return {
        'precision': round(precision_embeddings, 2),
        'recall': round(recall_embeddings, 2),
        'f1': round(f1_embeddings, 2),
    }
    

def test_nltk_tagger():
    return {
        'accuracy': round(accuracy_tnt_pos_tagger, 2)
    }

TESTS = [test_read_data_data_types, test_read_data_len_train_data, test_generate_vocabs, test_hmm, test_embeddings_model, test_nltk_tagger]

# Run tests and save results
res = {}
for test in TESTS:
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})

with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)

# Download the results.json file
# files.download('results.json')

####################